# Collaborative Filtering Recommendations using Normalized Values

In [484]:
import pandas as pd
import numpy as np
import implicit

from scipy.sparse import csr_matrix


In [485]:
train = pd.read_csv('data\\train-plays.csv')
test = pd.read_csv('data\\test-plays.csv')
game_coding = pd.read_csv('data\\game-coding.csv')


In [486]:
train.shape[0]
train

,amount,norm_amount,game_id,user_id
0,4.9,0.059036,0,0
1,62.0,0.300971,2,0
2,12.5,0.006572,6,1
3,24.0,0.058537,4,1
4,0.4,0.020000,17,1
...,...,...,...,...
39888,179.0,0.523810,1004,3402
39889,109.0,0.315476,1004,3403
39890,15.5,0.005906,1005,3404
39891,218.0,0.119798,1005,3405


## Format training data

In [487]:
game_user = train.pivot(
    index='game_id', columns='user_id', values='norm_amount')

In [488]:
game_user = game_user.fillna(0)
game_user

user_id,0,1,2,3,4,5,6,7,8,9,...,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406
game_id,,,,,,,,,,,,,,,,,,,,,
0,0.059036,0.009639,0.019277,0.118072,0.045783,0.001205,0.038554,0.038554,0.00241,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.007018,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.300971,0.000000,0.135922,0.155340,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.019208,0.001760,0.000000,0.001906,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.058537,0.001463,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1029,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1030,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [489]:
game_user_sparse = csr_matrix(game_user)

In [490]:
user_game = game_user.T
user_game

game_id,0,1,2,3,4,5,6,7,8,9,...,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.059036,0.000000,0.300971,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.009639,0.000000,0.000000,0.019208,0.058537,0.084942,0.006572,0.0,0.138191,0.03412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.019277,0.007018,0.135922,0.001760,0.001463,0.000000,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.118072,0.000000,0.155340,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.045783,0.000000,0.000000,0.001906,0.000000,0.005019,0.000473,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3403,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [491]:
user_game_sparse = csr_matrix(user_game)

In [492]:
# Number of possible interactions in the matrix
matrix_size = game_user_sparse.shape[0] * game_user_sparse.shape[1]
# Number of items interacted with
num_played = len(game_user_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_played / matrix_size))
sparsity

98.86683387172529

# Alternating Least Squares

In [493]:
als = implicit.als.AlternatingLeastSquares(128, 0.05, iterations=50)

In [494]:
als.fit(game_user_sparse)

100%|██████████| 50/50 [00:01<00:00, 28.11it/s]


In [495]:
game_recs = np.ndarray.flatten(als.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({'user_id': user_10, 'game_id': game_recs})
recommendations

100%|██████████| 3407/3407 [00:00<00:00, 46672.02it/s]


,user_id,game_id
0,0,86
1,0,488
2,0,494
3,0,85
4,0,406
...,...,...
34065,3406,880
34066,3406,358
34067,3406,162
34068,3406,311


In [496]:
results = recommendations.merge(test, on='user_id')

In [497]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)
results

,user_id,game_id_x,amount,norm_amount,game_id_y,recommended
0,0,86,144.0,0.505263,1,0
1,0,488,144.0,0.505263,1,0
2,0,494,144.0,0.505263,1,0
3,0,85,144.0,0.505263,1,0
4,0,406,144.0,0.505263,1,0
...,...,...,...,...,...,...
34065,3406,880,0.4,0.006557,1030,0
34066,3406,358,0.4,0.006557,1030,0
34067,3406,162,0.4,0.006557,1030,0
34068,3406,311,0.4,0.006557,1030,0


In [498]:
# accuracy
results[results.recommended == 1].shape[0] / test.shape[0]

0.1561491047842677

In [499]:
# % of games recommended
np.unique(game_recs).size / train.game_id.unique().size

0.8334946757018393

# Bayesian Personalized Ranking

In [500]:
bpr = implicit.bpr.BayesianPersonalizedRanking(128, 0.01, 0.05, iterations=200)

In [501]:
bpr.fit(game_user_sparse)

100%|██████████| 200/200 [00:03<00:00, 59.31it/s, train_auc=87.78%, skipped=9.24%]


In [502]:
game_recs = np.ndarray.flatten(bpr.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({'user_id':user_10, 'game_id':game_recs})

100%|██████████| 3407/3407 [00:00<00:00, 42587.54it/s]


In [503]:
results = recommendations.merge(test, on='user_id')
results

,user_id,game_id_x,amount,norm_amount,game_id_y
0,0,85,144.0,0.505263,1
1,0,36,144.0,0.505263,1
2,0,94,144.0,0.505263,1
3,0,180,144.0,0.505263,1
4,0,449,144.0,0.505263,1
...,...,...,...,...,...
34065,3406,689,0.4,0.006557,1030
34066,3406,672,0.4,0.006557,1030
34067,3406,647,0.4,0.006557,1030
34068,3406,535,0.4,0.006557,1030


In [504]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)

In [505]:
# accuracy
results[results.recommended == 1].shape[0] / test.shape[0]

0.2518344584678603

In [506]:
# % of games recommended
np.unique(game_recs).size / train.game_id.unique().size

0.8489835430784124

# Logistic Matrix Factorization

In [507]:
lmf = implicit.lmf.LogisticMatrixFactorization(128)

In [508]:
lmf.fit(game_user_sparse)

100%|██████████| 30/30 [00:06<00:00,  4.86it/s]


In [509]:
game_recs = np.ndarray.flatten(lmf.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({'user_id':user_10, 'game_id':game_recs})

100%|██████████| 3407/3407 [00:00<00:00, 44517.81it/s]


In [510]:
results = recommendations.merge(test, on='user_id')

In [511]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)

In [512]:
# accuracy
results[results.recommended == 1].shape[0] / test.shape[0]

0.05077781039037276

In [513]:
# % of games recommended
np.unique(game_recs).size / train.game_id.unique().size

0.7579864472410455

# ALS + BPR

In [514]:
user_ids = train['user_id'].unique()

In [515]:
user_recs = dict()
for user_id in user_ids:
    res = set(idx for idx, score in als.recommend(user_id, user_game_sparse))
    user_recs[user_id] = res
user_recs


{0: {85, 86, 94, 107, 406, 459, 486, 488, 494, 554},
 1: {7, 34, 326, 358, 462, 489, 503, 811, 814, 860},
 2: {300, 358, 412, 416, 679, 681, 730, 760, 773, 884},
 3: {82, 93, 289, 351, 496, 512, 594, 694, 753, 794},
 4: {65, 295, 296, 373, 384, 400, 469, 497, 707, 832},
 5: {131, 167, 178, 250, 302, 316, 388, 486, 868, 899},
 6: {62, 133, 180, 188, 201, 237, 523, 635, 695, 853},
 7: {85, 103, 138, 277, 305, 466, 510, 801, 821, 855},
 8: {85, 328, 341, 452, 721, 924, 931, 945, 949, 1015},
 9: {11, 34, 207, 336, 466, 637, 716, 781, 855, 884},
 10: {85, 86, 165, 253, 400, 407, 445, 461, 640, 939},
 11: {20, 27, 32, 55, 65, 340, 433, 649, 689, 902},
 12: {129, 152, 155, 305, 601, 619, 748, 828, 915, 1020},
 13: {46, 241, 319, 358, 478, 489, 658, 764, 815, 876},
 14: {147, 148, 149, 192, 274, 499, 794, 813, 921, 931},
 15: {21, 85, 86, 253, 274, 354, 358, 400, 420, 488},
 16: {165, 253, 257, 309, 363, 408, 461, 489, 640, 939},
 17: {90, 194, 248, 363, 439, 450, 505, 511, 829, 876},
 18: {75

In [516]:
for user_id in user_ids:
    res = set(idx for idx, score in bpr.recommend(user_id, user_game_sparse))
    als_res = user_recs[user_id]
    for r in res:
        als_res.add(r)
    user_recs[user_id] = als_res
user_recs


{0: {36,
  46,
  85,
  86,
  94,
  107,
  180,
  228,
  270,
  406,
  439,
  449,
  450,
  459,
  486,
  488,
  494,
  554},
 1: {7,
  34,
  283,
  285,
  326,
  358,
  457,
  462,
  489,
  503,
  643,
  756,
  793,
  811,
  814,
  860},
 2: {281,
  297,
  300,
  358,
  370,
  387,
  399,
  412,
  416,
  465,
  491,
  570,
  621,
  679,
  681,
  730,
  760,
  773,
  827,
  884},
 3: {82,
  93,
  289,
  308,
  351,
  369,
  449,
  491,
  496,
  512,
  516,
  518,
  594,
  618,
  694,
  752,
  753,
  794},
 4: {65,
  157,
  158,
  295,
  296,
  357,
  373,
  377,
  384,
  400,
  468,
  469,
  497,
  531,
  543,
  604,
  706,
  707,
  755,
  832},
 5: {85,
  131,
  163,
  164,
  167,
  178,
  206,
  250,
  287,
  302,
  316,
  324,
  325,
  346,
  388,
  486,
  629,
  805,
  868,
  899},
 6: {2,
  21,
  62,
  131,
  133,
  162,
  180,
  188,
  194,
  201,
  228,
  237,
  459,
  523,
  635,
  644,
  695,
  773,
  853},
 7: {7,
  8,
  11,
  85,
  103,
  138,
  163,
  277,
  305,
  466,
  51

In [517]:
num_total = 0
num_correct = 0
for user_id in user_ids:
    test_df = test[test['user_id'] == user_id]
    test_game_id = test_df['game_id'][user_id]
    num_total = num_total + 1
    if test_game_id in user_recs[user_id]:
        num_correct = num_correct + 1

num_correct / num_total


0.2967420017610801